In [15]:
from dotenv import load_dotenv

load_dotenv()

OSError: Starting path not found

In [ ]:
import os
os.getenv('PGDB_USER')

In [ ]:
%run -i '../pyscript/db/main.py'

In [16]:
from sqlalchemy import create_engine, text

In [17]:
sql = "select * from information_schema.tables where table_schema = 'daily_stats' and table_name ilike '%%_flex%%';"

df_tables = db.query_df(sql)

In [18]:
sql = "select * from information_schema.columns where table_schema = 'daily_stats' and table_name ilike '%%_flex%%';"

df_columns = db.query_df(sql)

In [19]:
sql = """select column_name from information_schema.columns 
where table_name ilike 'dc%'
and table_name ilike '%_eutran%' 
and table_name ilike '%_flex%'
and column_name ilike '%_nom' 
order by column_name;"""

df_kpi = db.query_df(sql)
kpis = df_kpi.values.tolist()   

In [20]:
kpis.sort()
kpis

[['dl_16qam_nom'],
 ['dl_16qam_nom'],
 ['dl_256qam_nom'],
 ['dl_256qam_nom'],
 ['dl_64qam_nom'],
 ['dl_64qam_nom'],
 ['dl_cell_throughput_nom'],
 ['dl_cell_throughput_nom'],
 ['dl_data_volume_nom'],
 ['dl_data_volume_nom'],
 ['dl_qpsk_nom'],
 ['dl_qpsk_nom'],
 ['dl_user_throughput_nom'],
 ['dl_user_throughput_nom'],
 ['e-rab_setup_success_rate_nom'],
 ['e-rab_setup_success_rate_nom'],
 ['erab_drop_call_rate_nom'],
 ['erab_drop_call_rate_nom'],
 ['intrafreq_hosr_nom'],
 ['intrafreq_hosr_nom'],
 ['packet_loss_(dl)_nom'],
 ['packet_loss_(dl)_nom'],
 ['packet_loss_(ul)_nom'],
 ['packet_loss_(ul)_nom'],
 ['ul_16qam_nom'],
 ['ul_16qam_nom'],
 ['ul_256qam_nom'],
 ['ul_256qam_nom'],
 ['ul_64qam_nom'],
 ['ul_64qam_nom'],
 ['ul_bler_nom'],
 ['ul_bler_nom'],
 ['ul_cell_throughput_nom'],
 ['ul_cell_throughput_nom'],
 ['ul_data_volume_nom'],
 ['ul_data_volume_nom'],
 ['ul_qpsk_nom'],
 ['ul_qpsk_nom'],
 ['ul_user_throughput_nom'],
 ['ul_user_throughput_nom']]

In [21]:
kpis = [kpi[0] for kpi in kpis]
kpis

['dl_16qam_nom',
 'dl_16qam_nom',
 'dl_256qam_nom',
 'dl_256qam_nom',
 'dl_64qam_nom',
 'dl_64qam_nom',
 'dl_cell_throughput_nom',
 'dl_cell_throughput_nom',
 'dl_data_volume_nom',
 'dl_data_volume_nom',
 'dl_qpsk_nom',
 'dl_qpsk_nom',
 'dl_user_throughput_nom',
 'dl_user_throughput_nom',
 'e-rab_setup_success_rate_nom',
 'e-rab_setup_success_rate_nom',
 'erab_drop_call_rate_nom',
 'erab_drop_call_rate_nom',
 'intrafreq_hosr_nom',
 'intrafreq_hosr_nom',
 'packet_loss_(dl)_nom',
 'packet_loss_(dl)_nom',
 'packet_loss_(ul)_nom',
 'packet_loss_(ul)_nom',
 'ul_16qam_nom',
 'ul_16qam_nom',
 'ul_256qam_nom',
 'ul_256qam_nom',
 'ul_64qam_nom',
 'ul_64qam_nom',
 'ul_bler_nom',
 'ul_bler_nom',
 'ul_cell_throughput_nom',
 'ul_cell_throughput_nom',
 'ul_data_volume_nom',
 'ul_data_volume_nom',
 'ul_qpsk_nom',
 'ul_qpsk_nom',
 'ul_user_throughput_nom',
 'ul_user_throughput_nom']

In [22]:
kpis = list(set(kpis))
kpis

['ul_cell_throughput_nom',
 'dl_data_volume_nom',
 'ul_bler_nom',
 'ul_256qam_nom',
 'ul_qpsk_nom',
 'packet_loss_(ul)_nom',
 'erab_drop_call_rate_nom',
 'ul_16qam_nom',
 'dl_16qam_nom',
 'dl_256qam_nom',
 'dl_cell_throughput_nom',
 'ul_user_throughput_nom',
 'intrafreq_hosr_nom',
 'dl_user_throughput_nom',
 'dl_qpsk_nom',
 'e-rab_setup_success_rate_nom',
 'packet_loss_(dl)_nom',
 'ul_data_volume_nom',
 'dl_64qam_nom',
 'ul_64qam_nom']

In [23]:
kpis.sort()

In [24]:
# join all elements in kpis separated by commas
kpis_string = ', \n'.join(['"'+ elem.replace('_nom', '') +'"' for elem in kpis])
print(kpis_string)


"dl_16qam", 
"dl_256qam", 
"dl_64qam", 
"dl_cell_throughput", 
"dl_data_volume", 
"dl_qpsk", 
"dl_user_throughput", 
"e-rab_setup_success_rate", 
"erab_drop_call_rate", 
"intrafreq_hosr", 
"packet_loss_(dl)", 
"packet_loss_(ul)", 
"ul_16qam", 
"ul_256qam", 
"ul_64qam", 
"ul_bler", 
"ul_cell_throughput", 
"ul_data_volume", 
"ul_qpsk", 
"ul_user_throughput"


In [ ]:
kpi_dict = {}

for table in df_columns['table_name'].unique():
    kpi_dict[table] = {}
    print()
    print(f'==========={table}==========')
    columns = df_columns[df_columns['table_name'] == table]['column_name'].unique()
    for column in columns:
        if (column.endswith('_nom')):
            nom = column
            kpi = column[:-4].strip()
            # find the corresponding denom
            denom = column[:-4] + '_den'
            # check denom column exists
            if denom in columns:
                # raise Exception(f'No denom column for {kpi}, nom is {nom}')
                # calculate kpi
                kpi_dict[table][kpi] = f'sum("{nom}")|||sum("{denom}")'
                print(kpi_dict[table][kpi], ' as ', '"'+kpi+'"', ',')
            else:
                kpi_dict[table][kpi] = f'sum("{nom}")'
                print(kpi_dict[table][kpi], ' as ', '"'+kpi+'"', ' ,')
        

# create tables

In [ ]:
dpm_path = '/home/hawkuser/data2/var/www/hawk-express-app/db/df_dpm/dnb_2023-4-26_6-53-52.json'

df_dpm = pd.read_json(dpm_path)
df_dpm.head()


In [ ]:
db.df_to_db(df_dpm, schema='daily_stats', name='df_dpm', if_exists='replace', index=False)

In [ ]:
cell_mapping_path = '/home/hawkuser/data2/var/www/hawk-express-app/db/df_dpm/cell_mapping_2023-4-26_14-12-24.json'

df_cell_mapping = pd.read_json(cell_mapping_path)
df_cell_mapping.head()


In [ ]:
db.df_to_db(df_cell_mapping, schema='daily_stats', name='cell_mapping', if_exists='replace', index=False)